# --- Day 16: Proboscidea Volcanium ---



In [61]:
def read_input(filename):
    flow_rates = dict()
    graph = dict()
    with open(filename) as infile:
        for line in infile:
            line = line.strip().replace("valves", "valve")
            line = line.strip().replace("tunnels lead", "tunnel leads")
            valve = line.split(" has flow rate=")[0].replace("Valve ", "")
            flow_rate, connections = line.split(" has flow rate=")[1].split(
                "; tunnel leads to valve "
            )
            flow_rates[valve] = int(flow_rate)
            graph[valve] = connections.split(", ")

    return flow_rates, graph


f, g = read_input("inputs/day16-example.txt")


In [ ]:
# The key is the order in which you open the valves
# The number of these is the number of permutations 
# Only open positive flow valves - if there are N with flow > 0
# then there are N! permutations

# Input data has N = 15 so N! = O(10^12) ...

# Need to decrease search space first

# We can open at most 15 valves in 30 minutes but this is assuming
# we can move directly between all positive flow rate valves

# The search space is the list of all positive flow rate valves that
# can be opened in 30 minutes

In [65]:
from queue import Queue
import numpy as np


def get_valve_distances(G, root):
    # Breadth-first search
    # https://en.wikipedia.org/wiki/Breadth-first_search
    #
    # Input: A graph G and a starting vertex root of G
    # Output: Goal state. The parent links trace the shortest path back to root[8]

    # let Q be a queue
    Q = Queue()

    # Dict to store distances from root
    dist = {k: np.inf for k in G.keys()}

    # label root as explored
    explored = set()
    explored.update({0})

    # Q.enqueue(root)
    Q.put(root)

    while not Q.empty():
        # v := Q.dequeue()
        v = Q.get()

        # if v is the goal then return v
        if v == root:
            dist[v] = 0

        # for all edges from v to w in G.adjacentEdges(v)
        for w in G[v]:
            if w not in explored:
                if dist[w] > dist[v] + 1:
                    dist[w] = dist[v] + 1
                Q.put(w)

                # label w as explored
                explored.update({w})

    return dist

get_valve_distances(g, "AA")


{'AA': 0,
 'BB': 1,
 'CC': 2,
 'DD': 1,
 'EE': 2,
 'FF': 3,
 'GG': 4,
 'HH': 5,
 'II': 1,
 'JJ': 2}